In [1]:
from qwen_finetuning import QwenFineTuningConfig, QwenFineTuning

In [2]:
config = QwenFineTuningConfig(
    model_name="Qwen/Qwen3-8B",
    train_file="data/train.jsonl",
    output_dir="./results_optimized",
    
    # Batch configuration - GOOD ✓
    batch_size=1,                    # Appropriate for 24GB VRAM
    gradient_accumulation_steps=16,  # Effective batch size = 16 ✓
    
    # Learning rate - GOOD ✓
    learning_rate=2e-4,              # Optimal for LoRA according to research
    
    # Training duration - CONSIDER INCREASE
    num_epochs=2,                    # Increased from 2 to 3 for 90K samples
    
    # Sequence length - GOOD ✓
    max_length=512,                  # Sufficient for Italian Q&A
    
    # LoRA hyperparameters - MOSTLY GOOD
    lora_r=16,                       # Good rank ✓
    lora_alpha=32,                   # Follows 2*r rule ✓  
    lora_dropout=0.1,                # IMPROVED: Increased from 0.05 to 0.1
    
    # CRITICAL ADDITIONS
    max_grad_norm=1.0,               # ADDED: Gradient clipping for stability
    
    # Performance optimizations - GOOD ✓
    dataset_num_proc=4,
    dataloader_num_workers=4,
    torch_empty_cache_steps=4,
)

In [3]:
config.print_config()


✓ Configuration set
Model: Qwen/Qwen3-8B
Learning rate: 0.0002
Max gradient norm: 1.0
Batch size: 1
Effective batch size: 16
Dataset processing cores: 4
Cache writer batch size: 500
DataLoader workers: 4
DataLoader optimizations: pin_memory=True, persistent_workers=True
GPU cache management: empty every 4 steps
Training stability: gradient clipping enabled at 1.0


In [4]:
# Create fine-tuning instance
finetuner = QwenFineTuning(config)


✓ Environment loaded, HF token available


In [5]:
# Load training data
train_data = finetuner.load_jsonl(config.train_file)


In [6]:
finetuner.run_complete_finetuning(train_data=train_data)


Train Dataset: 86929 examples
Categories: unknown(86929)
Answer distribution: A(24205), B(24441), C(24625), D(11146), E(2512)
Loading model and tokeniser...


config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

...

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


✓ Trainer configured with gradient clipping for improved stability
Starting training with gradient clipping enabled...


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
20,2.407700
40,1.610200
60,1.277300
80,1.144000
100,1.153100
120,1.130800
140,1.120800
160,1.103600
180,1.102300
200,1.106600


Saving model...
✓ Training completed


In [7]:
print(f"\n1-epoch fine-tuning completed successfully")
print(f"Model saved to: {config.output_dir}")


1-epoch fine-tuning completed successfully
Model saved to: ./results_optimized
